In [5]:
import glob
import pandas as pd
from shutil import copytree
import json
import pickle

In [6]:
with open('./../../examples/gene.json') as f:
    new_json = json.load(f)


In [19]:
# Copy E.coli essential genes from data directory into genes
#for directory in glob.glob('./../../data/**'):
    json_file = glob.glob('{}/*.json'.format(directory))
    with open(json_file[0]) as f:
        data = json.load(f)
    if "tags" in data:
        if "Escherichia coli" in data['tags']:
            copytree(directory, './../genes/{}'.format(data['gene_id']))

In [34]:
#for json_file in glob.glob('./../genes/**/*.json'):
    with open(json_file) as f:
        data = json.load(f)
    with open('./../../examples/gene.json') as f:
        new_json = json.load(f)
    
    new_json['gene_id'] = data['gene_id']
    new_json['part_type'] = data['info']['gene_metadata']['cloning']['part_type']
    new_json['tags'] = ['cds','moclo','Escherichia coli','essential']
    new_json['original_sequence'] = data['sequence']['original_sequence']
    new_json['status'] = 'Unapproved'
    new_json['author'] = [{'name': 'Keoni Gandall', 'email': 'koeng101@gmail.com', 'affiliation': 'BioBricks Foundation', 'orcid':'https://orcid.org/0000-0003-3995-6416'}]
    new_json['genbank'] = data['genbank']
    with open('./../genes/{}/{}.json'.format(data['gene_id'],data['gene_id']), 'w') as outfile:
        json.dump(new_json, outfile)

In [41]:
red20_map = {v: k.replace('U','T') for k, v in pickle.load(open("RED20_dict.pickle","rb")).items()}
red20_map['*'] = 'TGA'

orthogonal_primers = pd.read_csv('orthogonal_primers.csv')[['Forward','Reverse']]
orthogonal_primers['gene_id'] = ''
prefix = 'GGTCTCTA'
suffix = 'AGAGCTTCGAGACC'
for json_file in glob.glob('./../genes/**/*.json'):
    with open(json_file) as f:
        data = json.load(f)
    protein_seq = data['genbank']['translation'] + '*'
    dna_seq = []
    for aa in protein_seq:
        dna_seq.append(red20_map[aa])
    data['optimized_sequence'] = ''.join(dna_seq)
    for index,primer_pair in orthogonal_primers.iterrows():
        if primer_pair['gene_id'] == '':
            data['primer_for'] = primer_pair['Forward']
            data['primer_rev'] = primer_pair['Reverse']
            data['synthesized_sequence'] = primer_pair['Forward'] + prefix + optimized_seq + suffix + primer_pair['Reverse']
            orthogonal_primers.at[index,'gene_id'] = data['gene_id']
            break
    
    with open(json_file, 'w') as outfile:
        json.dump(data, outfile)

orthogonal_primers.to_csv('./onetube_ecoli.csv')

In [42]:
gene_ids = []
sequences = []
for json_file in glob.glob('./../genes/**/*.json'):
    with open(json_file) as f:
        data = json.load(f)
    gene_ids.append(data['gene_id'])
    sequences.append(data['synthesized_sequence'])
    
d = {
    "gene_id": gene_ids,
    "sequence": sequences
}
to_twist = pd.DataFrame.from_dict(d)
to_twist.to_csv('./to_twist.csv')